In [ ]:
import pandas as pd

CLIENT_ID = "Q7MgjgqFkM2Rn0ey3wk3"
CLIENT_SECRET = "o3W3lV_KFC"

# 이전에 작성한 search_book 함수를 여기에 추가
def search_book(query):

    from urllib.request import Request, urlopen

    from urllib.parse import urlencode, quote

    import json



    request = Request('https://openapi.naver.com/v1/search/book_adv?d_isbn='+quote(query))

    request.add_header('X-Naver-Client-Id', CLIENT_ID)

    request.add_header('X-Naver-Client-Secret', CLIENT_SECRET)



    response = urlopen(request).read().decode('utf-8')

    search_result = json.loads(response)

    return search_result


def get_description_from_isbn(isbn):
    try:
        book_info_list = search_book(isbn)['items']  # 'items'의 리스트를 가져옴

        if book_info_list:
            first_book_info = book_info_list[0]  # 첫 번째 아이템 선택

            # 첫 번째 아이템에서 'description'을 가져옴
            if 'description' in first_book_info:
                return first_book_info['description']
            else:
                return "Description not available"

        else:
            return "No book information found for the ISBN"

    except Exception as e:
        print(f"Failed to retrieve description for ISBN {isbn}: {e}")
        return None


# CSV 파일에서 ISBN 정보 읽기
def get_descriptions_from_csv(file_path):
    data = pd.read_csv(file_path)

    # ISBN 열에서 ISBN 정보 추출
    isbns = data['ISBN']

    descriptions = {}
    for isbn in isbns:
        isbn = int(isbn)
        description = get_description_from_isbn(str(isbn))
        if description:
            descriptions[isbn] = description

    return descriptions

if __name__ == "__main__":
    file_path = '2019-2022.csv'  # 여기에 파일 경로를 넣어주세요
    isbn_descriptions = get_descriptions_from_csv(file_path)

    # for isbn, description in isbn_descriptions.items():
    #     print(f"ISBN: {isbn}, Description: {description}")

    # DataFrame으로 CSV 파일 읽기
    data = pd.read_csv(file_path)

    # 새로운 열 추가
    data['Description'] = data['ISBN'].map(isbn_descriptions)

    # 수정된 DataFrame을 파일로 저장
    new_file_path = '2019-2022_with_description.csv'
    data.to_csv(new_file_path, index=False)
